In [1]:
from langchain_community.document_loaders.pdf import PyPDFLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter

paths = [
    "../documents/ApostilaLangChain.pdf",
    "../documents/Explorando a API da OpenAI.pdf",
    "../documents/Explorando o Universo das IAs com Hugging Face.pdf"
]

pages = []

for path in paths:
    loader = PyPDFLoader(path)
    pages.extend(loader.load())
    
splitter = RecursiveCharacterTextSplitter(
    chunk_size=500, 
    chunk_overlap=50,
    separators=["\n\n", "\n",  " ", ""]
)

documents = splitter.split_documents(pages)

/Users/viniciusfinger/Library/Python/3.9/lib/python/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


### Modificando metadata

In [2]:
for i, doc in enumerate(documents):
    doc.metadata["source"] = doc.metadata["source"].replace("../documents/", "")
    doc.metadata["id"] = i # Cria um id para o documento, poderia ser qualquer coisa.
    print(doc.metadata)

{'source': 'ApostilaLangChain.pdf', 'page': 0, 'id': 0}
{'source': 'ApostilaLangChain.pdf', 'page': 1, 'id': 1}
{'source': 'ApostilaLangChain.pdf', 'page': 1, 'id': 2}
{'source': 'ApostilaLangChain.pdf', 'page': 1, 'id': 3}
{'source': 'ApostilaLangChain.pdf', 'page': 1, 'id': 4}
{'source': 'ApostilaLangChain.pdf', 'page': 1, 'id': 5}
{'source': 'ApostilaLangChain.pdf', 'page': 1, 'id': 6}
{'source': 'ApostilaLangChain.pdf', 'page': 1, 'id': 7}
{'source': 'ApostilaLangChain.pdf', 'page': 2, 'id': 8}
{'source': 'ApostilaLangChain.pdf', 'page': 2, 'id': 9}
{'source': 'ApostilaLangChain.pdf', 'page': 2, 'id': 10}
{'source': 'ApostilaLangChain.pdf', 'page': 2, 'id': 11}
{'source': 'ApostilaLangChain.pdf', 'page': 2, 'id': 12}
{'source': 'ApostilaLangChain.pdf', 'page': 2, 'id': 13}
{'source': 'ApostilaLangChain.pdf', 'page': 2, 'id': 14}
{'source': 'ApostilaLangChain.pdf', 'page': 2, 'id': 15}
{'source': 'ApostilaLangChain.pdf', 'page': 3, 'id': 16}
{'source': 'ApostilaLangChain.pdf', 'page

In [3]:
from langchain_openai import OpenAIEmbeddings
from langchain_community.vectorstores.chroma import Chroma

embedding = OpenAIEmbeddings()

chroma_directory = "../documents/chroma_vectorstore"

vector_store = Chroma.from_documents(
    documents=documents,
    embedding=embedding,
    persist_directory=chroma_directory
)

### Busca Semântica
É o tipo de busca mais simples que tem, traz trechos duplicados, com muita similaridade e pouca informação adicional entre documentos.

In [4]:
question = 'O que é a OpenAI?'

documents_result = vector_store.similarity_search(question, k=3)

for doc in documents_result:
    print(doc.page_content)
    print(f'==========={doc.metadata}\n\n')

Explorando a API da OpenAI
01. Bem-vindos ao curso Explorando a API da OpenAI
Olá e bem-vindos ao nosso curso “Explorando a API da OpenAI”! Estou super animado por ter vocês
aqui e mal posso esperar para explorarmos juntos esse universo fascinante da programação e da
inteligência artificial.
Vocês já devem ter percebido que a IA está mudando o jogo em muitos campos, certo? É incrível como
ela está transformando a maneira como lidamos com informações e realiza tarefas que antes pareciam
==========={'id': 242, 'page': 4, 'source': 'Explorando a API da OpenAI.pdf'}


Explorando a API da OpenAI
01. Bem-vindos ao curso Explorando a API da OpenAI
Olá e bem-vindos ao nosso curso “Explorando a API da OpenAI”! Estou super animado por ter vocês
aqui e mal posso esperar para explorarmos juntos esse universo fascinante da programação e da
inteligência artificial.
Vocês já devem ter percebido que a IA está mudando o jogo em muitos campos, certo? É incrível como
ela está transformando a maneira como

In [5]:
question = 'O que a apostila de huggingface fala sobre OpenAI e chatGPT?'

documents_result = vector_store.similarity_search(question, k=3)

for doc in documents_result:
    print(doc.page_content)
    print(f'==========={doc.metadata}\n\n')

Explorando o Universo das IAs com Hugging Face
06. Modelos de conversação e geração de texto (chatbots)
Vamos aprender agora a usar modelos de conversação, que funcionam de forma parecida com chatbots
como ChatGPT . Na realidade, estes modelos estão classificados na tarefa de geração de texto ( text
generation ) no Hugging Face:
Figure 16: Página principal da tarefa text-generation do Hugging Face.
Vamos utilizar o modelo Felladrin/Llama-68M-Chat-v1 , disponível no link a seguir:
==========={'id': 573, 'page': 29, 'source': 'Explorando o Universo das IAs com Hugging Face.pdf'}


Explorando o Universo das IAs com Hugging Face
06. Modelos de conversação e geração de texto (chatbots)
Vamos aprender agora a usar modelos de conversação, que funcionam de forma parecida com chatbots
como ChatGPT . Na realidade, estes modelos estão classificados na tarefa de geração de texto ( text
generation ) no Hugging Face:
Figure 16: Página principal da tarefa text-generation do Hugging Face.
Vamos utiliz

No segundo dchunk busca erroneamente no "Explorando a API da OpenAI.pdf" porque não leva em consideração os metadados na hora da busca.